# Python basics: working with data

this notebook covers basic Python operations for data manipulation using `pandas`.

**dataset:** `data.csv` contains simulated health data with the following columns:
- `id`: unique identifier
- `zip_code`: Seattle area ZIP code  
- `age`: age in years
- `smoking`: binary variable (0 = non-smoker, 1 = smoker)
- `years_smoked`: number of years smoked (NA if non-smoker)

## 1. setup: import libraries

first, we import the libraries we'll use. Run this cell with `Shift+Enter`.

In [1]:
import pandas as pd
import numpy as np

## 2. reading data

use `pd.read_csv()` to read in a CSV file. this creates a **dataframe**, which is like an R dataframe.

In [2]:
# read the data
df = pd.read_csv("data.csv")

# view the first few rows
df.head()

,id,zip_code,age,smoking,years_smoked
0,1,98103,55,0,NaN
1,2,98104,75,1,9.0
2,3,98101,38,0,NaN
3,4,98122,91,0,NaN
4,5,98106,66,1,9.0


In [3]:
# check the shape (rows, columns)
print(f"Shape: {df.shape}")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

Shape: (1000, 5)
Number of rows: 1000
Number of columns: 5


In [4]:
# view column names
df.columns

Index(['id', 'zip_code', 'age', 'smoking', 'years_smoked'], dtype='object')

In [5]:
# view data types
df.dtypes

id                int64
zip_code          int64
age               int64
smoking           int64
years_smoked    float64
dtype: object

## 3. selecting columns

there are several ways to select columns in pandas:

In [6]:
# select a single column (returns a Series)
ages = df['age']
print(type(ages))
ages.head()

<class 'pandas.core.series.Series'>


0    55
1    75
2    38
3    91
4    66
Name: age, dtype: int64

In [7]:
# select multiple columns (returns a DataFrame)
subset = df[['id', 'age', 'smoking']]
print(type(subset))
subset.head()

<class 'pandas.core.frame.DataFrame'>


,id,age,smoking
0,1,55,0
1,2,75,1
2,3,38,0
3,4,91,0
4,5,66,1


## 4. basic statistics

pandas has built-in methods for common statistics:

In [8]:
# summary statistics for all numeric columns
df.describe()

,id,zip_code,age,smoking,years_smoked
count,1000.000000,1000.000000,1000.000000,1000.000000,484.000000
mean,500.500000,98131.120000,62.797000,0.484000,4.787190
std,288.819436,28.623817,21.954266,0.499994,3.189359
min,1.000000,98101.000000,25.000000,0.000000,0.000000
25%,250.750000,98108.000000,44.000000,0.000000,2.000000
50%,500.500000,98121.000000,62.000000,0.000000,5.000000
75%,750.250000,98146.000000,82.000000,1.000000,7.250000
max,1000.000000,98199.000000,100.000000,1.000000,10.000000


In [9]:
# individual statistics
print(f"mean age: {df['age'].mean():.2f}")
print(f"median age: {df['age'].median():.2f}")
print(f"std dev of age: {df['age'].std():.2f}")
print(f"min age: {df['age'].min()}")
print(f"max age: {df['age'].max()}")

mean age: 62.80
median age: 62.00
std dev of age: 21.95
min age: 25
max age: 100


In [10]:
# count values
df['smoking'].value_counts()

smoking
0    516
1    484
Name: count, dtype: int64

## 5. filtering rows

use boolean conditions to filter rows:

In [11]:
# filter to only smokers
smokers = df[df['smoking'] == 1]
print(f"number of smokers: {len(smokers)}")
smokers.head()

number of smokers: 484


,id,zip_code,age,smoking,years_smoked
1,2,98104,75,1,9.0
4,5,98106,66,1,9.0
8,9,98144,49,1,8.0
12,13,98178,96,1,3.0
14,15,98134,31,1,9.0


In [12]:
# filter with multiple conditions (use & for AND, | for OR)
older_smokers = df[(df['smoking'] == 1) & (df['age'] > 65)]
print(f"number of smokers over 65: {len(older_smokers)}")
older_smokers.head()

number of smokers over 65: 213


,id,zip_code,age,smoking,years_smoked
1,2,98104,75,1,9.0
4,5,98106,66,1,9.0
12,13,98178,96,1,3.0
25,26,98154,98,1,1.0
31,32,98125,77,1,8.0


In [13]:
# filter using .query() method (cleaner syntax)
older_smokers = df.query("smoking == 1 and age > 65")
older_smokers.head()

,id,zip_code,age,smoking,years_smoked
1,2,98104,75,1,9.0
4,5,98106,66,1,9.0
12,13,98178,96,1,3.0
25,26,98154,98,1,1.0
31,32,98125,77,1,8.0


## 6. creating new variables

add new columns by assigning to a new column name:

In [15]:
# create a copy so we don't modify the original
df_modified = df.copy()

# create a new column
df_modified['age_group'] = np.where(df_modified['age'] >= 65, 'senior', 'non-senior')
df_modified.head(10)

,id,zip_code,age,smoking,years_smoked,age_group
0,1,98103,55,0,NaN,non-senior
1,2,98104,75,1,9.0,senior
2,3,98101,38,0,NaN,non-senior
3,4,98122,91,0,NaN,senior
4,5,98106,66,1,9.0,senior
5,6,98178,74,0,NaN,senior
6,7,98112,67,0,NaN,senior
7,8,98107,38,0,NaN,non-senior
8,9,98144,49,1,8.0,non-senior
9,10,98164,93,0,NaN,senior


In [16]:
# create a column from arithmetic
df_modified['age_decades'] = df_modified['age'] / 10
df_modified[['age', 'age_decades']].head()

,age,age_decades
0,55,5.5
1,75,7.5
2,38,3.8
3,91,9.1
4,66,6.6


In [17]:
# create multiple categories with pd.cut()
df_modified['age_category'] = pd.cut(
    df_modified['age'], 
    bins=[0, 40, 60, 80, 100],
    labels=['Young', 'Middle', 'Senior', 'Elderly']
)
df_modified['age_category'].value_counts()

age_category
Middle     275
Elderly    263
Senior     258
Young      204
Name: count, dtype: int64

## 7. renaming columns

In [ ]:
# rename a column
df_renamed = df.rename(columns={'age': 'age_in_years'})
df_renamed.head()

,id,zip_code,age_in_years,smoking,years_smoked
0,1,98103,55,0,NaN
1,2,98104,75,1,9.0
2,3,98101,38,0,NaN
3,4,98122,91,0,NaN
4,5,98106,66,1,9.0
...,...,...,...,...,...
995,996,98195,26,0,NaN
996,997,98118,30,0,NaN
997,998,98125,79,1,9.0
998,999,98112,72,1,6.0


In [20]:
# rename multiple columns at once
df_renamed = df.rename(columns={
    'age': 'age_in_years',
    'smoking': 'is_smoker'
})
df_renamed.head()

,id,zip_code,age_in_years,is_smoker,years_smoked
0,1,98103,55,0,NaN
1,2,98104,75,1,9.0
2,3,98101,38,0,NaN
3,4,98122,91,0,NaN
4,5,98106,66,1,9.0


## 8. handling missing values

In [21]:
# check for missing values
df.isna().sum()

id                0
zip_code          0
age               0
smoking           0
years_smoked    516
dtype: int64

In [22]:
# filter to rows with missing years_smoked
missing_years = df[df['years_smoked'].isna()]
print(f"rows with missing years_smoked: {len(missing_years)}")
missing_years.head()

rows with missing years_smoked: 516


,id,zip_code,age,smoking,years_smoked
0,1,98103,55,0,NaN
2,3,98101,38,0,NaN
3,4,98122,91,0,NaN
5,6,98178,74,0,NaN
6,7,98112,67,0,NaN


In [23]:
# drop rows with any missing values
df_complete = df.dropna()
print(f"original rows: {len(df)}")
print(f"complete cases: {len(df_complete)}")

original rows: 1000
complete cases: 484


In [24]:
# fill missing values
df_filled = df.copy()
df_filled['years_smoked'] = df_filled['years_smoked'].fillna(0)
df_filled.isna().sum()

id              0
zip_code        0
age             0
smoking         0
years_smoked    0
dtype: int64

## 9. grouping and aggregating

In [25]:
# group by smoking status and calculate mean age
df.groupby('smoking')['age'].mean()

smoking
0    63.812016
1    61.714876
Name: age, dtype: float64

In [26]:
# multiple aggregations
df.groupby('smoking')['age'].agg(['mean', 'median', 'std', 'count'])

,mean,median,std,count
smoking,,,,
0,63.812016,63.0,21.742481,516
1,61.714876,61.0,22.149065,484


In [28]:
# group by multiple columns
df_modified = df.copy()
df_modified['age_group'] = np.where(df_modified['age'] >= 65, 'senior', 'non-senior')
df_modified.groupby(['smoking', 'age_group']).size()

smoking  age_group 
0        non-senior    270
         senior        246
1        non-senior    264
         senior        220
dtype: int64

## 10. sorting

In [29]:
# sort by age (ascending)
df.sort_values('age').head()

,id,zip_code,age,smoking,years_smoked
924,925,98101,25,1,2.0
110,111,98146,25,0,NaN
174,175,98164,25,1,1.0
505,506,98177,25,0,NaN
457,458,98154,25,0,NaN


In [30]:
# sort by age (descending)
df.sort_values('age', ascending=False).head()

,id,zip_code,age,smoking,years_smoked
292,293,98103,100,0,NaN
747,748,98117,100,0,NaN
19,20,98116,100,0,NaN
936,937,98199,100,0,NaN
905,906,98115,100,1,3.0


In [31]:
# sort by multiple columns
df.sort_values(['smoking', 'age'], ascending=[True, False]).head(10)

,id,zip_code,age,smoking,years_smoked
19,20,98116,100,0,NaN
37,38,98134,100,0,NaN
78,79,98144,100,0,NaN
254,255,98195,100,0,NaN
292,293,98103,100,0,NaN
322,323,98174,100,0,NaN
521,522,98199,100,0,NaN
747,748,98117,100,0,NaN
870,871,98116,100,0,NaN
936,937,98199,100,0,NaN


---
# practice problems

try these exercises on your own!

### Problem 1

Calculate the mean age of smokers vs non-smokers. Which group is older on average?

In [32]:
# Your code here
mean_age_smokers = df[df['smoking'] == 1]['age'].mean()
mean_age_non_smokers = df[df['smoking'] == 0]['age'].mean()
print(f"mean age of smokers: {mean_age_smokers:.2f}")
print(f"mean age of non-smokers: {mean_age_non_smokers:.2f}")

mean age of smokers: 61.71
mean age of non-smokers: 63.81


### Problem 2

Create a new column called `pack_years` that equals `years_smoked * 1.5` for smokers and `0` for non-smokers.

In [ ]:
# Your code here
df['pack_years'] = np.where(df['smoking'] == 1, df['years_smoked'] * 1.5, 0)
df.head()

,id,zip_code,age,smoking,years_smoked,pack_years
0,1,98103,55,0,NaN,NaN
1,2,98104,75,1,9.0,13.5
2,3,98101,38,0,NaN,NaN
3,4,98122,91,0,NaN,NaN
4,5,98106,66,1,9.0,13.5


### Problem 3

Find the 5 most common ZIP codes in the dataset.

In [34]:
# Your code here
top_zips = df['zip_code'].value_counts().head(5)
top_zips

zip_code
98102    44
98133    43
98195    42
98121    41
98125    40
Name: count, dtype: int64

### Problem 4

Filter the data to only include people aged 30-50 who are smokers, then calculate the mean `years_smoked` for this group.

In [35]:
# Your code here
middle_aged_df = df[(df['age'] >= 30) & (df['age'] <= 50) & (df['smoking'] == 1)]
mean_years_smoked = middle_aged_df['years_smoked'].mean()
print(f"mean years smoked for middle-aged smokers: {mean_years_smoked:.2f}")


mean years smoked for middle-aged smokers: 5.21


### Problem 5

Create a summary table showing the count and mean age by ZIP code, sorted by count (most to least).

In [37]:
# Your code here
summary_table = df.groupby('zip_code')['age'].agg(['count', 'mean']).sort_values(by='count', ascending=False)
summary_table.head()

,count,mean
zip_code,,
98102,44,61.727273
98133,43,64.651163
98195,42,60.690476
98121,41,65.121951
98125,40,57.600000
